# $l_0$-constrained Ridge Regression

In this notebook, we show example usage of SCG on Sparse Ridge Regression, namely:
$$\underset{\|\boldsymbol{\beta}\|_0 \leq k}{\operatorname{min}} \frac{1}{n} \|\boldsymbol{X} \boldsymbol{\beta} - \boldsymbol{Y}\|_2^2 + \gamma \|\boldsymbol{\beta} \|_2^2$$
<!-- where we use $\tilde{\boldsymbol{\beta}}$ to denote the optimal solution.  -->

<!-- The data generation follows the procedure in [Bertsimas et al. (2019)](https://arxiv.org/pdf/1902.06547). Below are some quick links to different sections: 
- [Playground](#playground)
    - [Ground-truth Checking](#ground_truth_checking)
- [Formal Testing](#formal_testing)
- [Plotting](#plotting)
    - [Solved Instances](#sol_instance)
    - [Fixed Indices](#indices_fixed) -->

In [19]:
# === import packages ===
import sys 
sys.path.append("../..") 
from source import utils,env,main,test
import numpy as np
import pandas as pd
import random
import math
import time

----

 ## Generate Synthetic dataset 
<!-- This section conducts numerical experiments on synthetic datasets to demonstrate the efficiency of our proposed SCG method compared with the existing baseline, Safe Screening Rules (SSR) in the pre-processing step.  -->

In [21]:
# === Generate synthetic dataset ===
random_seed = None
num = 50
dim = 500
sparsity_ = 10
rho_ = 0.5
SNR_ = 3.5
(X,Y,beta_true) = env.sparse_generation(num = num,
                              dim = dim,
                              sparsity=sparsity_,
                              rho = rho_,
                              SNR = SNR_,
                              random_seed = random_seed)


gamma_ = 5

----

 ## Apply SCG to generate cuts

In [29]:
inc_max, inc_len = sparsity_, 2  # default setting for inclusive cuts generation
exc_max, exc_len = dim - sparsity_, 3 # default setting for exclusive cuts generation


# === start SCG to generate cuts ===
procedure = main.ridge_screen(covariate=X,response=Y,gamma = gamma_,sparsity = sparsity_)

procedure.get_feas_beta_ub(relax_solver='gurobi')


S_inc,Z_inc = procedure.inclusive_cuts(max_num = inc_max, max_len = inc_len)

S_exc,Z_exc = procedure.exclusive_cuts(max_num = exc_max, max_len = exc_len)

----

## Solve the problem with SCG

In [31]:
# === Solver settings ===
time_lim = 15 * 60
output_log = 0
ite_lim = None
gap_lim = 0.01


print('=== start solving with SCG ===')
current_time = time.time()
SCG_model = utils.ridge_train(X = X,Y = Y,
                          sparsity=sparsity_,gamma = gamma_,
                          Outputlog=output_log,mip_gap = gap_lim,
                          S_set = S_inc+S_exc, Z_set = Z_inc+Z_exc,
                          max_ite = ite_lim,max_min = time_lim)
end_time = time.time()
print('solving time:', end_time - current_time)
print('=== done ===')